In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("all_cricket_cleaned.csv")
df = df.drop(columns = ['Unnamed: 0']).set_index('index')
df.head()

,Team1,Team2,Winner,Margin,Ground,Scorecard,Match_Date,Match_Type,Month,Year
index,,,,,,,,,,
0,Australia,India,Australia,inns & 141 runs,Sydney,Test # 1481,"Jan,2,2000",Test,Jan,2000
1,South Africa,England,South Africa,inns & 37 runs,Cape Town,Test # 1482,"Jan,2,2000",Test,Jan,2000
2,South Africa,England,England,2 wickets,Centurion,Test # 1483,"Jan,14,2000",Test,Jan,2000
3,India,South Africa,South Africa,4 wickets,Mumbai,Test # 1484,"Feb,24,2000",Test,Feb,2000
4,Pakistan,Sri Lanka,Sri Lanka,2 wickets,Rawalpindi,Test # 1485,"Feb,26,2000",Test,Feb,2000


### Split data

In [3]:
X = df[["Team1", "Team2", "Ground", "Match_Type", "Month"]]
y = df["Winner"]  #.values.reshape(-1,1)
print(X.shape, y.shape)

(4364, 5) (4364,)


In [7]:
data = X.copy()

data_binary_encoded = pd.get_dummies(data, columns=["Team1", "Team2", "Ground", "Match_Type", "Month"])
data_binary_encoded.head()

,Team1_Afghanistan,Team1_Africa XI,Team1_Asia XI,Team1_Australia,Team1_Bahrain,Team1_Bangladesh,Team1_Belgium,Team1_Belize,Team1_Bermuda,Team1_Botswana,...,Month_Dec,Month_Feb,Month_Jan,Month_Jul,Month_Jun,Month_Mar,Month_May,Month_Nov,Month_Oct,Month_Sep
index,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [8]:
from sklearn.model_selection import train_test_split

X = pd.get_dummies(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [9]:
X_train.head()

,Team1_Afghanistan,Team1_Africa XI,Team1_Asia XI,Team1_Australia,Team1_Bahrain,Team1_Bangladesh,Team1_Belgium,Team1_Belize,Team1_Bermuda,Team1_Botswana,...,Month_Dec,Month_Feb,Month_Jan,Month_Jul,Month_Jun,Month_Mar,Month_May,Month_Nov,Month_Oct,Month_Sep
index,,,,,,,,,,,,,,,,,,,,,
5,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
41,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
84,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


### Pre-processing

In [15]:
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train.shape

(3273, 273)

In [11]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)


In [13]:
from tensorflow.keras.utils import to_categorical

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
y_train_categorical.shape

(3273, 51)

### Deep Neural Net

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=600, activation='relu', input_dim=273))
model.add(Dense(units=400, activation='relu'))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=51, activation='softmax'))

In [33]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 600)               164400    
_________________________________________________________________
dense_10 (Dense)             (None, 400)               240400    
_________________________________________________________________
dense_11 (Dense)             (None, 200)               80200     
_________________________________________________________________
dense_12 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense_13 (Dense)             (None, 51)                5151      
Total params: 510,251
Trainable params: 510,251
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.compile(optimizer = 'Nadam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [40]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=150,
    shuffle=True,
    verbose=2
)


Epoch 1/150
3273/3273 - 1s - loss: 0.1819 - acc: 0.8796
Epoch 2/150
3273/3273 - 1s - loss: 0.1867 - acc: 0.8821
Epoch 3/150
3273/3273 - 1s - loss: 0.1947 - acc: 0.8879
Epoch 4/150
3273/3273 - 1s - loss: 0.1796 - acc: 0.8824
Epoch 5/150
3273/3273 - 1s - loss: 0.1976 - acc: 0.8802
Epoch 6/150
3273/3273 - 1s - loss: 0.1824 - acc: 0.8802
Epoch 7/150
3273/3273 - 1s - loss: 0.1849 - acc: 0.8842
Epoch 8/150
3273/3273 - 1s - loss: 0.1960 - acc: 0.8842
Epoch 9/150
3273/3273 - 1s - loss: 0.1918 - acc: 0.8808
Epoch 10/150
3273/3273 - 1s - loss: 0.1705 - acc: 0.8873
Epoch 11/150
3273/3273 - 1s - loss: 0.1683 - acc: 0.8842
Epoch 12/150
3273/3273 - 1s - loss: 0.1665 - acc: 0.8876
Epoch 13/150
3273/3273 - 1s - loss: 0.1663 - acc: 0.8860
Epoch 14/150
3273/3273 - 1s - loss: 0.1661 - acc: 0.8891
Epoch 15/150
3273/3273 - 1s - loss: 0.1667 - acc: 0.8925
Epoch 16/150
3273/3273 - 1s - loss: 0.1687 - acc: 0.8906
Epoch 17/150
3273/3273 - 1s - loss: 0.1666 - acc: 0.8863
Epoch 18/150
3273/3273 - 1s - loss: 0.16

3273/3273 - 1s - loss: 0.1613 - acc: 0.8903
Epoch 145/150
3273/3273 - 1s - loss: 0.1618 - acc: 0.8857
Epoch 146/150
3273/3273 - 1s - loss: 0.1612 - acc: 0.8870
Epoch 147/150
3273/3273 - 1s - loss: 0.1614 - acc: 0.8854
Epoch 148/150
3273/3273 - 1s - loss: 0.1617 - acc: 0.8879
Epoch 149/150
3273/3273 - 1s - loss: 0.1614 - acc: 0.8915
Epoch 150/150
3273/3273 - 1s - loss: 0.1612 - acc: 0.8909


In [41]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1091/1091 - 0s - loss: 6.3934 - acc: 0.5435
Deep Neural Network - Loss: 6.393417818627152, Accuracy: 0.5435380339622498


In [42]:
predictions_encoded = model.predict_classes(X_test_scaled[:5])
predictions_labels = label_encoder.inverse_transform(predictions_encoded)

/Users/shahzinakhan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [44]:
print("Encoded Predictions: " , predictions_encoded)
print("Prediction Labels: ", predictions_labels)
print("Actual Labels: ", list(y_test[:5]))

Encoded Predictions:  [ 3  0 45 45 16]
Prediction Labels:  ['Australia' 'Afghanistan' 'West Indies' 'West Indies' 'Ireland']
Actual Labels:  ['Australia', 'Afghanistan', 'Zimbabwe', 'West Indies', 'Ireland']
